In [9]:
import pandas as pd
import numpy as np
import datetime
import streamlit as st
from datetime import datetime
import seaborn as sns
import os
import altair as alt
import pydeck as pdk
from PIL import Image
import matplotlib.pyplot as plt
from scipy import stats 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib
import sklearn

from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

import lightgbm as lgb
from fbprophet import Prophet
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

#importing a scoring metric to compare methods
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import adfuller
%matplotlib inline


processed2 = pd.read_csv("../data/processed/processed.csv") #the .. goes one directory up (two periods)

In [10]:
processed2.measurement_date = pd.to_datetime(processed2.measurement_date, format = '%Y-%m-%d')
processed2.set_index(['measurement_date', 'user_code'], inplace=True) #reseting the index
processed2['distance_yds'] = (processed2['distance'] * 1.0936)

In [11]:
processed2['active_calories_burned_norm'] = preprocessing.scale(processed2['active_calories_burned'])
processed2['steps_count_norm'] = preprocessing.scale(processed2['steps_count'])
processed2['distance_norm'] = preprocessing.scale(processed2['distance'])
processed2['steps_speed_norm'] = preprocessing.scale(processed2['steps_speed'])
processed2['body_temperature_avg_norm'] = preprocessing.scale(processed2['body_temperature_avg'])
processed2['pulse_average_norm'] = preprocessing.scale(processed2['pulse_average'])
processed2['stand_hours_total_norm'] = preprocessing.scale(processed2['stand_hours_total'])
processed2['total_number_of_flights_climbed_norm'] = preprocessing.scale(processed2['total_number_of_flights_climbed'])
processed2['pulse_min_norm'] = preprocessing.scale(processed2['pulse_min'])
processed2['pulse_max_norm'] = preprocessing.scale(processed2['pulse_max'])
processed2['average_spo2_value_norm'] = preprocessing.scale(processed2['average_spo2_value'])
processed2['distance_mi_norm'] = preprocessing.scale(processed2['distance_mi'])
processed2['ACWR_norm'] = preprocessing.scale(processed2['ACWR'])
processed2['height_in_norm'] = preprocessing.scale(processed2['height_in'])
processed2['weight_lbs_norm'] = preprocessing.scale(processed2['weight_lbs'])
processed2['rmssd_norm'] = preprocessing.scale(processed2['rmssd'])
processed2['bpm_norm'] = preprocessing.scale(processed2['bpm'])
processed2['heart_rate_norm'] = preprocessing.scale(processed2['heart_rate'])
processed2['BMI_norm'] = preprocessing.scale(processed2['BMI'])
processed2['distance_yds_norm'] = preprocessing.scale(processed2['distance_yds'])

In [12]:
processed2.reset_index(inplace=True) #date needs to be an index before you can do test/train split
processed2.set_index(['measurement_date', 'user_code'], inplace=True) #reseting the index

In [13]:
processed2.over_train.mean() #about 10% in the overtraining class. 

0.096045197740113

In [14]:
#doing a train/test split
range_train = pd.date_range(start = '2020-01-01', end = '2020-04-16') #doing an 80/20 split
df_train = processed2.loc[range_train]

range_test = pd.date_range(start = '2020-04-17', end = '2020-05-13')
df_test = processed2.loc[range_test]

In [15]:
drop = ['steps_count',
       'active_calories_burned', 'distance', 'steps_speed',
       'body_temperature_avg', 'pulse_average', 'stand_hours_total',
       'total_number_of_flights_climbed', 'pulse_min', 'pulse_max',
       'average_spo2_value', 'distance_mi', 'ACWR', 'height', 'weight',
       'height_in', 'weight_lbs', 'rmssd', 'pnn50', 'bpm', 'lnrmssd',
       'is_resting', 'heart_rate', 'BMI', 'lnrmssd_z', 'ACWR_z','distance_yds', 'distance_norm']
#ONLY RUN THIS ONCE!
df_train.drop(drop, axis = 1, inplace=True)  #removing non-normalized features and other features that will not be included
df_test.drop(drop, axis = 1, inplace=True)

/Users/davidkirui/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
#testing RF after dropping HRV ::::
#del df_train['ACWR_norm'] #removing ACWR
del df_train['rmssd_norm'] #removing HRV
#del df_test['ACWR_norm'] #removing ACWR
del df_test['rmssd_norm'] #removing HRV

In [17]:
df_train.to_csv("../data/processed/df_train.csv")

In [18]:
df_test.to_csv("../data/processed/df_test.csv")

In [19]:
##converting data to arrays for RF
#labels are the values that we want to predict:
train_labels = df_train['over_train']

#remove the labels from the features
#axis 1 refers to the columns
train_features = df_train.drop('over_train', axis = 1)

#Saving feature names for later use
feature_list = list(train_features.columns)

#convert to numpy array
features_train = np.array(feature_list)

In [20]:
train_labels.to_csv("../data/processed/train_labels.csv")
train_features.to_csv("../data/processed/train_features.csv")

In [21]:
# Create the model with 100 trees

#setting random number seed
myfavoritenumber = 13
seed = myfavoritenumber
np.random.seed(seed)

model = RandomForestClassifier(n_estimators=200, 
                               bootstrap = True,
                               max_features = 'sqrt',
                               class_weight = {0:2,1:1}) 

In [22]:
model.fit(train_features, train_labels)

RandomForestClassifier(class_weight={0: 2, 1: 1}, max_features='sqrt',
                       n_estimators=200)

In [26]:
#Actual class predictions
##converting data to arrays for RF
#labels are the values that we want to predict:
test_labels = df_test['over_train']

#remove the labels from the features
#axis 1 refers to the columns
test_features = df_test.drop('over_train', axis = 1)


#Saving feature names for later use
feature_list = list(test_features.columns)

#convert to numpy array
features_test = np.array(test_features)

rf_predictions = model.predict(test_features)
rf_probs = model.predict_proba(test_features)[:,1]

In [27]:
test_labels.to_csv("../data/processed/test_labels.csv")
test_features.to_csv("../data/processed/test_features.csv")

In [28]:
from sklearn.metrics import roc_auc_score

In [29]:
# Calculate roc auc
roc_value = roc_auc_score(test_labels, rf_probs)
roc_value #HRV still in there. 
#0.8639485226223453 (HRV in, ACWR out)
#0.8811677631578948 (HRV out, ACWR in) <- this is interesting!  bc you don't need HRV to make predictions

0.8743002654662974

In [30]:
from sklearn.metrics import confusion_matrix
results = confusion_matrix(test_labels, rf_predictions) 

In [31]:
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(test_labels, rf_predictions)) 
print('Report : ')
print(classification_report(test_labels, rf_predictions))

Confusion Matrix :
[[569  39]
 [ 70  44]]
Accuracy Score : 0.8490304709141274
Report : 
              precision    recall  f1-score   support

           0       0.89      0.94      0.91       608
           1       0.53      0.39      0.45       114

    accuracy                           0.85       722
   macro avg       0.71      0.66      0.68       722
weighted avg       0.83      0.85      0.84       722

